# To unify the sampler, we borrow function from R.

In [60]:
repeats = int(10)
nfold = int(5)
nsample = int(240)
cf=int(nsample/nfold)
lf=nsample-cf
import rpy2.robjects as robjects
import numpy as np
setseed = robjects.r['set.seed']
sample = robjects.r['sample']
sd = robjects.r["sd"]

# Platform information

In [2]:
import platform
platform.uname()

uname_result(system='Windows', node='LAPTOP-9BDORFP1', release='10', version='10.0.17134', machine='AMD64', processor='Intel64 Family 6 Model 158 Stepping 9, GenuineIntel')

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13544685235067680755, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 78616985
 locality {
   bus_id: 1
 }
 incarnation: 6268415023482120726
 physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [7]:
import sys
print (sys.version)

3.5.4 | packaged by conda-forge | (default, Dec 18 2017, 06:53:03) [MSC v.1900 64 bit (AMD64)]


# Loading data

In [24]:
import pandas as pd
pheno = pd.read_csv('./2017heteroPheno.csv')
I = pd.read_table('./Hetero_realigned_cov10_filtered3.raw',sep="\s+").as_matrix()-1
bcw=pheno.bcw.as_matrix()
length=pheno.length.as_matrix()
AccSum1=np.reshape(np.zeros(repeats*nfold),(nfold,repeats))
AccSum2=np.copy(AccSum1);AccSum3=np.copy(AccSum1);AccSum4=np.copy(AccSum1)

# Import packages for deep learning models

In [46]:
from scipy.stats.stats import pearsonr
import keras 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
import time
print("version of tensorflow_gpu:",tf.__version__)
print("version of tensorflow_bakend_keras:",keras.__version__)

version of tensorflow_gpu: 1.4.0
version of tensorflow_bakend_keras: 2.1.4


# Neural network

In [47]:
start = time.time()
#10-repeat-5-fold Cross validation as same as R
for j in range(repeats):
    setseed(100+3*(j+1)+1)
    id =np.array(sample(robjects.IntVector(np.arange(1,(nsample+1)) %nfold)))
    for i in range(nfold):
        test = np.array(np.where(id==i)).reshape(cf)
        train = np.array(np.where(id!=i)).reshape(lf)
        #Model
        model = Sequential()
        model.add(Dense(200, activation='relu', input_dim=3928))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(1)) 
        #Optimizer
        model.compile(optimizer='Adam',loss='mse')
        #Feed the data
        model.fit(I[train,],bcw[train],epochs=35, batch_size=128)
        #Prediction
        pred = model.predict(I[test,],batch_size=None, verbose=0).reshape(cf)
        pred1 = model.predict(I[train,],batch_size=None, verbose=0).reshape(lf)
        #Accuracy
        AccSum1[i,j] = pearsonr(pred,bcw[test])[0]
        AccSum2[i,j] = pearsonr(pred1,bcw[train])[0]
end = time.time()

Epoch 1/35
192/192 [==============================] - 5s 25ms/step - loss: 271.0513
Epoch 2/35
192/192 [==============================] - 0s 228us/step - loss: 95.6780
Epoch 3/35
192/192 [==============================] - 0s 213us/step - loss: 136.5306
Epoch 4/35
192/192 [==============================] - 0s 213us/step - loss: 83.4220
Epoch 5/35
192/192 [==============================] - 0s 187us/step - loss: 79.8054
Epoch 6/35
192/192 [==============================] - 0s 234us/step - loss: 80.6101
Epoch 7/35
192/192 [==============================] - 0s 187us/step - loss: 70.7693
Epoch 8/35
192/192 [==============================] - 0s 244us/step - loss: 63.6917
Epoch 9/35
192/192 [==============================] - 0s 197us/step - loss: 62.9703
Epoch 10/35
192/192 [==============================] - 0s 192us/step - loss: 58.1840
Epoch 11/35
192/192 [==============================] - 0s 197us/step - loss: 52.1009
Epoch 12/35
192/192 [==============================] - 0s 197us/step - lo

192/192 [==============================] - 0s 145us/step - loss: 49.6146
Epoch 19/35
192/192 [==============================] - 0s 156us/step - loss: 49.0588
Epoch 20/35
192/192 [==============================] - 0s 156us/step - loss: 47.3177
Epoch 21/35
192/192 [==============================] - 0s 182us/step - loss: 44.3373
Epoch 22/35
192/192 [==============================] - 0s 161us/step - loss: 41.3722
Epoch 23/35
192/192 [==============================] - 0s 161us/step - loss: 39.5690
Epoch 24/35
192/192 [==============================] - 0s 171us/step - loss: 38.0662
Epoch 25/35
192/192 [==============================] - 0s 156us/step - loss: 36.0688
Epoch 26/35
192/192 [==============================] - 0s 140us/step - loss: 33.7659
Epoch 27/35
192/192 [==============================] - 0s 156us/step - loss: 31.9321
Epoch 28/35
192/192 [==============================] - 0s 140us/step - loss: 30.3199
Epoch 29/35
192/192 [==============================] - 0s 166us/step - loss: 

192/192 [==============================] - 0s 119us/step - loss: 28.3281
Epoch 1/35
192/192 [==============================] - 5s 24ms/step - loss: 305.4917
Epoch 2/35
192/192 [==============================] - 0s 125us/step - loss: 105.7457
Epoch 3/35
192/192 [==============================] - 0s 109us/step - loss: 121.4300
Epoch 4/35
192/192 [==============================] - 0s 124us/step - loss: 129.7501
Epoch 5/35
192/192 [==============================] - 0s 114us/step - loss: 79.4852
Epoch 6/35
192/192 [==============================] - 0s 130us/step - loss: 76.5974
Epoch 7/35
192/192 [==============================] - 0s 120us/step - loss: 87.4758
Epoch 8/35
192/192 [==============================] - 0s 130us/step - loss: 77.1753
Epoch 9/35
192/192 [==============================] - 0s 109us/step - loss: 62.1521
Epoch 10/35
192/192 [==============================] - 0s 119us/step - loss: 61.4798
Epoch 11/35
192/192 [==============================] - 0s 120us/step - loss: 61.061

192/192 [==============================] - 0s 140us/step - loss: 29.6494
Epoch 18/35
192/192 [==============================] - 0s 218us/step - loss: 26.2615
Epoch 19/35
192/192 [==============================] - 0s 161us/step - loss: 23.0512
Epoch 20/35
192/192 [==============================] - 0s 114us/step - loss: 20.1090
Epoch 21/35
192/192 [==============================] - 0s 140us/step - loss: 17.3020
Epoch 22/35
192/192 [==============================] - 0s 125us/step - loss: 14.6815
Epoch 23/35
192/192 [==============================] - 0s 140us/step - loss: 12.7552
Epoch 24/35
192/192 [==============================] - 0s 125us/step - loss: 9.8825
Epoch 25/35
192/192 [==============================] - 0s 146us/step - loss: 8.9935
Epoch 26/35
192/192 [==============================] - 0s 120us/step - loss: 6.9310
Epoch 27/35
192/192 [==============================] - 0s 109us/step - loss: 5.7277
Epoch 28/35
192/192 [==============================] - 0s 119us/step - loss: 4.39

Epoch 35/35
192/192 [==============================] - 0s 120us/step - loss: 1.9919
Epoch 1/35
192/192 [==============================] - 4s 22ms/step - loss: 306.1880
Epoch 2/35
192/192 [==============================] - 0s 120us/step - loss: 128.7218
Epoch 3/35
192/192 [==============================] - 0s 114us/step - loss: 90.3142
Epoch 4/35
192/192 [==============================] - 0s 114us/step - loss: 120.6236
Epoch 5/35
192/192 [==============================] - 0s 114us/step - loss: 97.9710
Epoch 6/35
192/192 [==============================] - 0s 114us/step - loss: 72.6289
Epoch 7/35
192/192 [==============================] - 0s 130us/step - loss: 71.7981
Epoch 8/35
192/192 [==============================] - 0s 120us/step - loss: 79.2461
Epoch 9/35
192/192 [==============================] - 0s 109us/step - loss: 77.3684
Epoch 10/35
192/192 [==============================] - 0s 125us/step - loss: 66.1018
Epoch 11/35
192/192 [==============================] - 0s 114us/step - lo

Epoch 18/35
192/192 [==============================] - 0s 135us/step - loss: 41.0181
Epoch 19/35
192/192 [==============================] - 0s 125us/step - loss: 36.6676
Epoch 20/35
192/192 [==============================] - 0s 125us/step - loss: 34.0011
Epoch 21/35
192/192 [==============================] - 0s 130us/step - loss: 31.2300
Epoch 22/35
192/192 [==============================] - 0s 125us/step - loss: 26.7679
Epoch 23/35
192/192 [==============================] - 0s 119us/step - loss: 25.5305
Epoch 24/35
192/192 [==============================] - 0s 135us/step - loss: 21.6664
Epoch 25/35
192/192 [==============================] - 0s 124us/step - loss: 19.6439
Epoch 26/35
192/192 [==============================] - 0s 135us/step - loss: 16.7437
Epoch 27/35
192/192 [==============================] - 0s 161us/step - loss: 14.7029
Epoch 28/35
192/192 [==============================] - 0s 130us/step - loss: 12.2970
Epoch 29/35
192/192 [==============================] - 0s 130us/s

192/192 [==============================] - 0s 135us/step - loss: 10.1638
Epoch 1/35
192/192 [==============================] - 5s 25ms/step - loss: 301.7558
Epoch 2/35
192/192 [==============================] - 0s 125us/step - loss: 116.4169
Epoch 3/35
192/192 [==============================] - 0s 148us/step - loss: 112.0548
Epoch 4/35
192/192 [==============================] - 0s 119us/step - loss: 128.4498
Epoch 5/35
192/192 [==============================] - 0s 125us/step - loss: 89.1006
Epoch 6/35
192/192 [==============================] - 0s 130us/step - loss: 74.4173
Epoch 7/35
192/192 [==============================] - 0s 125us/step - loss: 84.5924
Epoch 8/35
192/192 [==============================] - 0s 125us/step - loss: 82.1927
Epoch 9/35
192/192 [==============================] - 0s 120us/step - loss: 67.5127
Epoch 10/35
192/192 [==============================] - 0s 130us/step - loss: 61.0363
Epoch 11/35
192/192 [==============================] - 0s 114us/step - loss: 65.351

192/192 [==============================] - 0s 151us/step - loss: 45.4128
Epoch 18/35
192/192 [==============================] - 0s 135us/step - loss: 42.7604
Epoch 19/35
192/192 [==============================] - 0s 182us/step - loss: 40.3333
Epoch 20/35
192/192 [==============================] - 0s 224us/step - loss: 38.3880
Epoch 21/35
192/192 [==============================] - 0s 146us/step - loss: 35.6290
Epoch 22/35
192/192 [==============================] - 0s 156us/step - loss: 33.4366
Epoch 23/35
192/192 [==============================] - 0s 130us/step - loss: 31.4345
Epoch 24/35
192/192 [==============================] - 0s 146us/step - loss: 28.7907
Epoch 25/35
192/192 [==============================] - 0s 146us/step - loss: 27.0745
Epoch 26/35
192/192 [==============================] - 0s 166us/step - loss: 24.8389
Epoch 27/35
192/192 [==============================] - 0s 145us/step - loss: 22.2607
Epoch 28/35
192/192 [==============================] - 0s 140us/step - loss: 

Epoch 9/35
192/192 [==============================] - 0s 135us/step - loss: 62.7399
Epoch 10/35
192/192 [==============================] - 0s 135us/step - loss: 63.0644
Epoch 11/35
192/192 [==============================] - 0s 140us/step - loss: 59.0397
Epoch 12/35
192/192 [==============================] - 0s 151us/step - loss: 50.9637
Epoch 13/35
192/192 [==============================] - 0s 166us/step - loss: 48.4630
Epoch 14/35
192/192 [==============================] - 0s 145us/step - loss: 45.2015
Epoch 15/35
192/192 [==============================] - 0s 140us/step - loss: 40.3646
Epoch 16/35
192/192 [==============================] - 0s 161us/step - loss: 38.4718
Epoch 17/35
192/192 [==============================] - 0s 151us/step - loss: 36.7889
Epoch 18/35
192/192 [==============================] - 0s 140us/step - loss: 32.3382
Epoch 19/35
192/192 [==============================] - 0s 145us/step - loss: 29.8248
Epoch 20/35
192/192 [==============================] - 0s 124us/st

Epoch 27/35
192/192 [==============================] - 0s 130us/step - loss: 31.4257
Epoch 28/35
192/192 [==============================] - 0s 156us/step - loss: 29.6744
Epoch 29/35
192/192 [==============================] - 0s 135us/step - loss: 27.2562
Epoch 30/35
192/192 [==============================] - 0s 187us/step - loss: 25.4882
Epoch 31/35
192/192 [==============================] - 0s 140us/step - loss: 23.9961
Epoch 32/35
192/192 [==============================] - 0s 125us/step - loss: 22.1363
Epoch 33/35
192/192 [==============================] - 0s 140us/step - loss: 20.1307
Epoch 34/35
192/192 [==============================] - 0s 140us/step - loss: 18.6261
Epoch 35/35
192/192 [==============================] - 0s 130us/step - loss: 16.8996
Epoch 1/35
192/192 [==============================] - 6s 32ms/step - loss: 270.9280
Epoch 2/35
192/192 [==============================] - 0s 182us/step - loss: 105.5440
Epoch 3/35
192/192 [==============================] - 0s 140us/ste

In [48]:
print("eplased time",end - start)

eplased time 414.90176010131836


In [49]:
#validate on test and train group
print(np.mean(AccSum1),np.mean(AccSum2))

0.300227714854724 0.9741363391166052


In [8]:
from keras.utils import plot_model
plot_model(model, to_file='modelbasic.png',show_layer_names=True,show_shapes=True)

# Multi task deep learning model

In [23]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers

In [30]:
#Optimizer
rmsprop=optimizers.RMSprop(lr=0.003)

In [31]:
start = time.time()
#10-repeat-5-fold Cross validation as same as R
for j in range(repeats):
    setseed(100+3*(j+1)+1)
    id =np.array(sample(robjects.IntVector(np.arange(1,(nsample+1)) %nfold)))
    for i in range(nfold):
        test = np.array(np.where(id==i)).reshape(cf)
        train = np.array(np.where(id!=i)).reshape(lf)
        #Model
        main_input = Input(shape=(3928,), name='main_input')
        x = Dense(200, activation='relu')(main_input)


        x1 = Dense(20, activation='relu')(x)
        y1 = Dense(1,name='y1')(x1)

        x2 = Dense(100, activation='relu')(x)
        y2= Dense(1,name='y2')(x2)

        model = Model(inputs=[main_input], outputs=[y1,y2])
        #Compile
        model.compile(optimizer=rmsprop,loss={'y1': 'mse','y2':'mse'},loss_weights={'y1': 1.,'y2': 0.2})
        #Train
        model.fit({'main_input': I[train,]},{'y1': bcw[train],'y2': length[train]},
                      epochs=30, batch_size=128,callbacks=None)
        #Accuracy
        AccSum3[i,j] =pearsonr(model.predict(I[test,], batch_size=None, verbose=0)[0].reshape(cf),bcw[test])[0]
        AccSum4[i,j] =pearsonr(model.predict(I[train,], batch_size=None, verbose=0)[0].reshape(lf),bcw[train])[0]
end = time.time()

Epoch 1/30
192/192 [==============================] - 3s 15ms/step - loss: 3681.0228 - y1_loss: 3173.5425 - y2_loss: 2537.4023
Epoch 2/30
192/192 [==============================] - 0s 135us/step - loss: 239.1185 - y1_loss: 233.1516 - y2_loss: 29.8346
Epoch 3/30
192/192 [==============================] - 0s 223us/step - loss: 98.7264 - y1_loss: 97.8493 - y2_loss: 4.3859
Epoch 4/30
192/192 [==============================] - 0s 172us/step - loss: 81.9072 - y1_loss: 77.6582 - y2_loss: 21.2452
Epoch 5/30
192/192 [==============================] - 0s 151us/step - loss: 75.5747 - y1_loss: 74.7296 - y2_loss: 4.2251
Epoch 6/30
192/192 [==============================] - 0s 161us/step - loss: 72.6947 - y1_loss: 71.8664 - y2_loss: 4.1412
Epoch 7/30
192/192 [==============================] - 0s 145us/step - loss: 66.6175 - y1_loss: 66.0963 - y2_loss: 2.6063
Epoch 8/30
192/192 [==============================] - 0s 156us/step - loss: 62.3524 - y1_loss: 62.0120 - y2_loss: 1.7020
Epoch 9/30
192/192 [==

192/192 [==============================] - 0s 353us/step - loss: 48.8611 - y1_loss: 45.6616 - y2_loss: 15.9975
Epoch 16/30
192/192 [==============================] - 0s 125us/step - loss: 38.1060 - y1_loss: 37.8119 - y2_loss: 1.4704
Epoch 17/30
192/192 [==============================] - 0s 114us/step - loss: 32.3161 - y1_loss: 32.1580 - y2_loss: 0.7902
Epoch 18/30
192/192 [==============================] - 0s 125us/step - loss: 32.3970 - y1_loss: 32.1967 - y2_loss: 1.0019
Epoch 19/30
192/192 [==============================] - 0s 114us/step - loss: 64.8154 - y1_loss: 64.1331 - y2_loss: 3.4114
Epoch 20/30
192/192 [==============================] - 0s 156us/step - loss: 178.5529 - y1_loss: 177.3131 - y2_loss: 6.1992
Epoch 21/30
192/192 [==============================] - 0s 151us/step - loss: 56.6753 - y1_loss: 56.4748 - y2_loss: 1.0025
Epoch 22/30
192/192 [==============================] - 0s 109us/step - loss: 28.7707 - y1_loss: 28.1697 - y2_loss: 3.0049
Epoch 23/30
192/192 [============

192/192 [==============================] - 0s 125us/step - loss: 20.8600 - y1_loss: 20.7253 - y2_loss: 0.6736
Epoch 30/30
192/192 [==============================] - 0s 114us/step - loss: 34.3969 - y1_loss: 34.2067 - y2_loss: 0.9508
Epoch 1/30
192/192 [==============================] - 3s 14ms/step - loss: 818.1457 - y1_loss: 613.0870 - y2_loss: 1025.2935
Epoch 2/30
192/192 [==============================] - 0s 125us/step - loss: 255.6204 - y1_loss: 245.3458 - y2_loss: 51.3728
Epoch 3/30
192/192 [==============================] - 0s 125us/step - loss: 84.5319 - y1_loss: 84.3342 - y2_loss: 0.9882
Epoch 4/30
192/192 [==============================] - 0s 145us/step - loss: 74.6838 - y1_loss: 74.4251 - y2_loss: 1.2933
Epoch 5/30
192/192 [==============================] - 0s 135us/step - loss: 73.7829 - y1_loss: 72.1783 - y2_loss: 8.0230
Epoch 6/30
192/192 [==============================] - 0s 125us/step - loss: 69.5077 - y1_loss: 67.9779 - y2_loss: 7.6490
Epoch 7/30
192/192 [===============

192/192 [==============================] - 0s 114us/step - loss: 58.9859 - y1_loss: 56.8311 - y2_loss: 10.7739
Epoch 13/30
192/192 [==============================] - 0s 130us/step - loss: 55.5208 - y1_loss: 53.3670 - y2_loss: 10.7691
Epoch 14/30
192/192 [==============================] - 0s 119us/step - loss: 51.0561 - y1_loss: 49.8054 - y2_loss: 6.2534
Epoch 15/30
192/192 [==============================] - 0s 120us/step - loss: 52.9326 - y1_loss: 52.2644 - y2_loss: 3.3412
Epoch 16/30
192/192 [==============================] - 0s 125us/step - loss: 54.6028 - y1_loss: 54.3250 - y2_loss: 1.3891
Epoch 17/30
192/192 [==============================] - 0s 125us/step - loss: 47.0957 - y1_loss: 46.6415 - y2_loss: 2.2710
Epoch 18/30
192/192 [==============================] - 0s 146us/step - loss: 41.2536 - y1_loss: 41.0675 - y2_loss: 0.9307
Epoch 19/30
192/192 [==============================] - 0s 125us/step - loss: 36.3441 - y1_loss: 36.0900 - y2_loss: 1.2701
Epoch 20/30
192/192 [=============

192/192 [==============================] - 0s 130us/step - loss: 25.5443 - y1_loss: 25.2634 - y2_loss: 1.4047
Epoch 27/30
192/192 [==============================] - 0s 114us/step - loss: 39.5646 - y1_loss: 38.4806 - y2_loss: 5.4197
Epoch 28/30
192/192 [==============================] - 0s 125us/step - loss: 32.2251 - y1_loss: 31.0350 - y2_loss: 5.9501
Epoch 29/30
192/192 [==============================] - 0s 130us/step - loss: 44.8319 - y1_loss: 39.8139 - y2_loss: 25.0901
Epoch 30/30
192/192 [==============================] - 0s 119us/step - loss: 37.9423 - y1_loss: 37.7930 - y2_loss: 0.7469
Epoch 1/30
192/192 [==============================] - 3s 16ms/step - loss: 2629.1631 - y1_loss: 2418.6618 - y2_loss: 1052.5063
Epoch 2/30
192/192 [==============================] - 0s 125us/step - loss: 260.5872 - y1_loss: 259.5406 - y2_loss: 5.2328
Epoch 3/30
192/192 [==============================] - 0s 135us/step - loss: 172.5092 - y1_loss: 172.1366 - y2_loss: 1.8631
Epoch 4/30
192/192 [========

...

Epoch 14/30
192/192 [==============================] - 0s 146us/step - loss: 75.6722 - y1_loss: 74.9280 - y2_loss: 3.7209
Epoch 15/30
192/192 [==============================] - 0s 140us/step - loss: 51.4471 - y1_loss: 50.5970 - y2_loss: 4.2502
Epoch 16/30
192/192 [==============================] - 0s 275us/step - loss: 38.9266 - y1_loss: 38.6015 - y2_loss: 1.6252
Epoch 17/30
192/192 [==============================] - 0s 135us/step - loss: 36.5160 - y1_loss: 36.2968 - y2_loss: 1.0960
Epoch 18/30
192/192 [==============================] - 0s 140us/step - loss: 41.8491 - y1_loss: 41.5699 - y2_loss: 1.3959
Epoch 19/30
192/192 [==============================] - 0s 145us/step - loss: 41.2705 - y1_loss: 40.6318 - y2_loss: 3.1931
Epoch 20/30
192/192 [==============================] - 0s 125us/step - loss: 52.1334 - y1_loss: 51.2163 - y2_loss: 4.5856
Epoch 21/30
192/192 [==============================] - 0s 119us/step - loss: 50.2132 - y1_loss: 49.1556 - y2_loss: 5.2882
Epoch 22/30
192/192 [===

192/192 [==============================] - 0s 124us/step - loss: 17.8412 - y1_loss: 16.7685 - y2_loss: 5.3634
Epoch 28/30
192/192 [==============================] - 0s 125us/step - loss: 14.7542 - y1_loss: 13.6930 - y2_loss: 5.3060
Epoch 29/30
192/192 [==============================] - 0s 171us/step - loss: 60.3822 - y1_loss: 59.5753 - y2_loss: 4.0345
Epoch 30/30
192/192 [==============================] - 0s 125us/step - loss: 24.3971 - y1_loss: 22.2479 - y2_loss: 10.7460
Epoch 1/30
192/192 [==============================] - 4s 22ms/step - loss: 1622.3161 - y1_loss: 1290.5751 - y2_loss: 1658.7049
Epoch 2/30
192/192 [==============================] - 0s 146us/step - loss: 126.5796 - y1_loss: 121.8221 - y2_loss: 23.7873
Epoch 3/30
192/192 [==============================] - 0s 135us/step - loss: 91.2283 - y1_loss: 83.4903 - y2_loss: 38.6899
Epoch 4/30
192/192 [==============================] - 0s 135us/step - loss: 78.7928 - y1_loss: 77.3229 - y2_loss: 7.3491
Epoch 5/30
192/192 [=========

Epoch 4/30
192/192 [==============================] - 0s 130us/step - loss: 88.0571 - y1_loss: 87.4034 - y2_loss: 3.2686
Epoch 5/30
192/192 [==============================] - 0s 125us/step - loss: 84.2316 - y1_loss: 82.2501 - y2_loss: 9.9076
Epoch 6/30
192/192 [==============================] - 0s 125us/step - loss: 84.1301 - y1_loss: 82.2627 - y2_loss: 9.3369
Epoch 7/30
192/192 [==============================] - 0s 120us/step - loss: 73.3679 - y1_loss: 72.9709 - y2_loss: 1.9850
Epoch 8/30
192/192 [==============================] - 0s 140us/step - loss: 69.5564 - y1_loss: 68.9101 - y2_loss: 3.2318
Epoch 9/30
192/192 [==============================] - 0s 124us/step - loss: 70.5547 - y1_loss: 69.5282 - y2_loss: 5.1324
Epoch 10/30
192/192 [==============================] - 0s 130us/step - loss: 73.9216 - y1_loss: 72.0101 - y2_loss: 9.5575
Epoch 11/30
192/192 [==============================] - 0s 114us/step - loss: 73.3590 - y1_loss: 70.8154 - y2_loss: 12.7182
Epoch 12/30
192/192 [========

192/192 [==============================] - 0s 161us/step - loss: 46.2209 - y1_loss: 45.9858 - y2_loss: 1.1755
Epoch 19/30
192/192 [==============================] - 0s 120us/step - loss: 51.6801 - y1_loss: 49.9016 - y2_loss: 8.8923
Epoch 20/30
192/192 [==============================] - 0s 125us/step - loss: 47.0960 - y1_loss: 45.6043 - y2_loss: 7.4588
Epoch 21/30
192/192 [==============================] - 0s 124us/step - loss: 57.2180 - y1_loss: 56.9923 - y2_loss: 1.1285
Epoch 22/30
192/192 [==============================] - 0s 114us/step - loss: 38.7775 - y1_loss: 38.1438 - y2_loss: 3.1682
Epoch 23/30
192/192 [==============================] - 0s 130us/step - loss: 20.3575 - y1_loss: 19.5059 - y2_loss: 4.2577
Epoch 24/30
192/192 [==============================] - 0s 135us/step - loss: 19.0262 - y1_loss: 18.1886 - y2_loss: 4.1883
Epoch 25/30
192/192 [==============================] - 0s 125us/step - loss: 57.1440 - y1_loss: 56.5963 - y2_loss: 2.7382
Epoch 26/30
192/192 [===============

192/192 [==============================] - 4s 20ms/step - loss: 1713.2786 - y1_loss: 1483.6376 - y2_loss: 1148.2052
Epoch 2/30
192/192 [==============================] - 0s 125us/step - loss: 178.7211 - y1_loss: 177.1331 - y2_loss: 7.9400
Epoch 3/30
192/192 [==============================] - 0s 119us/step - loss: 81.1592 - y1_loss: 78.2046 - y2_loss: 14.7729
Epoch 4/30
192/192 [==============================] - 0s 130us/step - loss: 73.5838 - y1_loss: 73.3267 - y2_loss: 1.2852
Epoch 5/30
192/192 [==============================] - 0s 130us/step - loss: 66.9353 - y1_loss: 66.7529 - y2_loss: 0.9120
Epoch 6/30
192/192 [==============================] - 0s 125us/step - loss: 70.3433 - y1_loss: 70.1102 - y2_loss: 1.1654
Epoch 7/30
192/192 [==============================] - 0s 125us/step - loss: 81.8912 - y1_loss: 80.2195 - y2_loss: 8.3588
Epoch 8/30
192/192 [==============================] - 0s 115us/step - loss: 65.1301 - y1_loss: 62.8376 - y2_loss: 11.4627
Epoch 9/30
192/192 [=============

In [32]:
print("eplased time",end - start)

eplased time 312.6359119415283


In [33]:
#validate on test and train
print(np.mean(AccSum3),np.mean(AccSum4))

0.30666759049545633 0.9376196462345063


In [34]:
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_layer_names=False,show_shapes=True)

In [153]:
#save result
import pandas as pd
seven_models_r = pd.read_excel("All_models_Acc.xlsx")
v1=pd.DataFrame(pd.DataFrame(AccSum1).values.flatten(),columns=["NN"],index=range(1,51))
v2=pd.DataFrame(pd.DataFrame(AccSum3).values.flatten(),columns=["MNN"],index=range(1,51))
pd.concat([seven_models_r,v1,v2],axis=1).to_excel('All_models_Acc.xlsx')